In [1]:
import sys
import os

# add the parent directory ('BNetzA') to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np

from models import *
from Data_Preprocessing.data_preprocessing import *

import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

pd.set_option('display.float_format', '{:.4f}'.format)

# Loading Datasets

In [3]:
# load Data
path_to_excel = "../data/EVS4_20140118_dataV9.xlsx"
df = load_data(path_to_excel, sheet_name="DatasetV9")

In [4]:
# df_test_list = [df_test, df_test_xlog, df_test_xlog_ylog, df_test_ylog, df_test_agg, df_test_agg_log, df_test_non_agg, df_test_group_agg]
# baseline, xlog, xlog ylog, ylog, agg, agg ylog, non agg, group agg
# missing: agg xlog, agg xlog ylog
# missing: non agg xlog, non agg xlog ylog, non agg ylog
# missing: group agg xlog, group agg xlog ylog, group agg ylog

In [5]:
outcome_variables = ["cTOTEXn", "cTOTEXn", "cTOTEXn_log", "cTOTEXn_log", "cTOTEXn", "cTOTEXn_log", "cTOTEXn", "cTOTEXn"]
names = ["Baseline", "XLog", "XLog YLog", "YLog", "Aggregates Only", "Aggregates Only YLog", "Disaggregates Only", "Aggregates N1-4 N5-7"]
outcome_transformation = ["None", "None", "log", "log", "None", "log", "None", "None"]
random_states = [39, 40, 41, 42, 43]
columns = ["Model", "Training RMSE", "Training MAE", "Training MAPE", 
           "Testing RMSE", "Testing MAE", "Testing MAPE"]

# Lasso Regression

In [10]:
lasso_results = pd.DataFrame(columns=columns)
lasso_models = []
lasso_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lasso_result, lasso_model, lasso_vip = lasso_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Lasso {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lasso_results = pd.concat([lasso_results, lasso_result], axis=0, ignore_index=True)
        lasso_models.append(lasso_model)
        lasso_vips.append(lasso_vip)


In [11]:
lasso_results.loc[:, lasso_results.columns != 'Model'] = lasso_results.loc[:, lasso_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lasso_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Lasso Aggregates Only YLog,19509173.4220,7579512.9960,0.1240,11268863.2300,5562088.2800,0.1400
6,Lasso XLog YLog,20123969.0380,7400527.3200,0.1220,15132882.6880,6881961.5580,0.1480
1,Lasso Aggregates Only,7583134.1940,4640771.0300,0.1740,17908963.3640,7210410.5920,0.1960
4,Lasso Disaggregates Only,4387758.6500,2950621.7080,0.1460,16269428.3820,7617384.2360,0.3640
3,Lasso Baseline,4451456.6160,2969861.2360,0.1460,16244300.1740,7705064.8980,0.3680
0,Lasso Aggregates N1-4 N5-7,5383870.0300,3738333.9340,0.1680,15177457.6260,7541244.6360,0.4220
7,Lasso YLog,238548674.4180,31717017.5340,0.3220,1519404205.0400,349358998.6680,0.8500
5,Lasso XLog,72088678.5860,37785655.4460,1.6920,61913800.8180,36692748.2300,2.1400


# Linear Regression with selected Features from Lasso¶

In [7]:
lr_results = pd.DataFrame(columns=columns)
lr_models = []
lr_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        lr_result, lr_model, lr_vip = lasso_feature_selection_linear_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Linear Regresion {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        lr_results = pd.concat([lr_results, lr_result], axis=0, ignore_index=True)
        lr_models.append(lr_model)
        lr_vips.append(lr_vip)


In [22]:
lr_models[0].params

const                                              68537151.7000
yConnections.streetlights.N7                        5158308.6462
yConnections.cus.N4                                 2907069.2698
yConnections.cus.N5                                24909672.1438
yConnections.cus.N6                                  500464.1828
yInjectionPoints.other.N3                           2716976.6820
yInjectionPoints.other.N4                            244473.0988
yMeters.read.op.N7                                  9940495.8955
yMeters.read.ext.N6                                 -173180.8136
yMeters.cp.nonctrl.N5                                  2657.4683
yMeters.over10MWh.RPM.N3                            2694383.8295
yMeters.over10MWh.RPM.N7                             882378.7178
yMeters.others.N5                                   4822337.8700
yCables.circuit.N3                              -1612711694.7082
yCables.all.N13.sum                              1612781141.9656
yLines.excl.house.N7     

In [11]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,15662174.8340,6470256.8480,0.1200,10119945.7280,5421080.0660,0.1380
6,Linear Regresion XLog YLog,15695991.1580,6450477.1820,0.1160,14841106.8660,6892647.4480,0.1500
1,Linear Regresion Aggregates Only,6968612.1220,4517576.9640,0.1840,16672876.6480,6854655.0280,0.2040
4,Linear Regresion Disaggregates Only,3087245.3840,2234400.2060,0.1360,36073805.1720,14134950.3240,0.3700
0,Linear Regresion Aggregates N1-4 N5-7,4026616.5160,2995976.8780,0.1660,17920304.9960,7680720.8120,0.3880
3,Linear Regresion Baseline,3013376.3480,2182353.8140,0.1400,31828459.8400,12707307.4880,0.3940
7,Linear Regresion YLog,35254376.8480,11705541.8580,0.2320,2970443916.6660,674762907.3500,1.1100
5,Linear Regresion XLog,63012231.9180,37332414.5620,1.8200,84939751.7220,46865510.8120,2.8300


In [50]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,15662174.8340,6470256.8480,0.1200,10119945.7280,5421080.0660,0.1380
6,Linear Regresion XLog YLog,15695991.1580,6450477.1820,0.1160,14841106.8660,6892647.4480,0.1500
1,Linear Regresion Aggregates Only,6968612.1220,4517576.9640,0.1840,16672876.6480,6854655.0280,0.2040
4,Linear Regresion Disaggregates Only,3087245.3840,2234400.2060,0.1360,36073805.1720,14134950.3240,0.3700
3,Linear Regresion Baseline,3013376.3480,2182353.8140,0.1400,31828459.8400,12707307.4880,0.3940
0,Linear Regresion Aggregates N1-4 N5-7,4108654.4700,2992750.0760,0.1600,15963882.2620,7931386.7280,0.4320
7,Linear Regresion YLog,35432244.1120,11773861.0600,0.2320,2833722930.6120,644609710.0100,1.0960
5,Linear Regresion XLog,63012231.9180,37332414.5620,1.8200,84939751.7220,46865510.8120,2.8300


In [19]:
lr_models[5].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            cTOTEXn_log   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     635.3
Date:                Fri, 16 Aug 2024   Prob (F-statistic):          2.74e-135
Time:                        17:30:16   Log-Likelihood:                 87.058
No. Observations:                 174   AIC:                            -136.1
Df Residuals:                     155   BIC:                            -76.09
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   16.9784      0.012   1440.761      0.000      16.955      17.002
yConnections.streetlights.sum            0.0071      0.015      0.481      0.631      -0.022       0.036
yMeters.cp.ctrl.tot                      0.0100      0.014      0.728      0.468      -0.017       0.037
yMeters.active.tot                       0.0354      0.038      0.929      0.354      -0.040       0.111
yMeters.over10MWh.RPM.tot                0.0139      0.032      0.439      0.661      -0.048       0.076
yMeters.others.tot                       0.0171      0.016      1.076      0.284      -0.014       0.048
yNet.length.excl.house.tot               0.1183      0.069      1.706      0.090      -0.019       0.255
ySubstations.own.tot                    -0.0082      0.069     -0.119      0.906      -0.144       0.128
yInstalledPower.renewables.hydro.tot     0.0004      0.016      0.025      0.980      -0.032       0.033
yInstalledPower.other.tot                0.0454      0.016      2.890      0.004       0.014       0.076
yInstalledPower.KWKG.other.tot           0.0276      0.026      1.067      0.288      -0.023       0.079
yInjection.tot                          29.1962     23.589      1.238      0.218     -17.401      75.794
yEnergy.losses.tot                       0.4667      0.074      6.305      0.000       0.320       0.613
yEnergy.delivered.tot                  -29.0790     23.594     -1.232      0.220     -75.686      17.528
yEnergy.delivered.N1357.sum              0.4065      0.134      3.025      0.003       0.141       0.672
yEnergy.delivered.net.tot                0.0117      0.093      0.127      0.899      -0.171       0.195
yEnergy.delivered.net.N67.sum            0.0431      0.032      1.337      0.183      -0.021       0.107
yInjection.net.N5to7.sum                 0.0627      0.026      2.393      0.018       0.011       0.115
yPeakload.from.higher.nonsim.sum         0.0206      0.020      1.042      0.299      -0.018       0.060
==============================================================================
Omnibus:                        1.476   Durbin-Watson:                   2.225
Prob(Omnibus):                  0.478   Jarque-Bera (JB):                1.496
Skew:                          -0.218   Prob(JB):                        0.473
Kurtosis:                       2.874   Cond. No.                     9.37e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.37e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
lr_results.loc[:, lr_results.columns != 'Model'] = lr_results.loc[:, lr_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
lr_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
2,Linear Regresion Aggregates Only YLog,14677973.3300,6067041.1080,0.1140,9699432.5860,5298720.8000,0.1400
6,Linear Regresion XLog YLog,15345685.9140,6299959.0220,0.1140,15030208.1440,6867786.8940,0.1520
1,Linear Regresion Aggregates Only,6789600.6460,4405321.2560,0.1840,16359456.6640,6706496.7620,0.1980
3,Linear Regresion Baseline,2051665.5240,1587536.0200,0.1180,26143451.1540,10879766.3880,0.4840
4,Linear Regresion Disaggregates Only,2123288.3400,1621571.9800,0.1180,29745202.1300,12349881.1780,0.5460
0,Linear Regresion Aggregates N1-4 N5-7,3323075.7760,2508061.0320,0.1600,16217742.8820,8458170.1880,0.6380
5,Linear Regresion XLog,54432064.9480,34150405.9900,1.6960,89508940.9240,50923828.3020,2.4240
7,Linear Regresion YLog,26041250.9800,9175203.8540,0.1880,inf,10142320547350045689240233885590865790529548774...,24478134620635746182772844188168143481390764669...


# Random Forest Regression

In [14]:
rf_results = pd.DataFrame(columns=columns)
rf_models = []
rf_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        rf_result, rf_model, rf_vip = random_forest_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Random Forest {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        rf_results = pd.concat([rf_results, rf_result], axis=0, ignore_index=True)
        rf_models.append(rf_model)
        rf_vips.append(rf_vip)


Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                    Model Training RMSE Training MAE Training MAPE  \
0  Random Forest Baseline   30248617.33   6199937.38          0.09   

  Testing RMSE Testing MAE Testing MAPE  
0   7621079.89  4469284.64         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Random Forest XLog   30305327.84   6277681.97          0.09   7810148.75   

  Testing MAE Testing MAPE  
0  4522345.53         0.17  
Performing Random Forest...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Evaluating the model...


                     Model Training RMSE Training MAE Training MAPE  \
0  Random Forest XLog YLog   36440603.78   7026397.02          0.07   

  Testing RMSE Testing MAE Testing MAPE  
0   4776229.17  3354260.27  

In [15]:
rf_results.loc[:, rf_results.columns != 'Model'] = rf_results.loc[:, rf_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
rf_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
7,Random Forest YLog,36693302.4640,7167030.3120,0.0700,39793393.2180,12615968.8700,0.1820
6,Random Forest XLog YLog,36467920.7900,7086270.6680,0.0700,38545130.3440,12294038.1220,0.1860
2,Random Forest Aggregates Only YLog,37720341.4260,7369140.6220,0.0720,37824967.0540,11750373.7020,0.1920
0,Random Forest Aggregates N1-4 N5-7,29553159.6620,6340270.4580,0.0800,41580339.8440,13574560.1200,0.2220
1,Random Forest Aggregates Only,29774959.3140,6523046.7580,0.0820,38484881.3920,12521322.5200,0.2220
3,Random Forest Baseline,29180181.5740,6122334.1160,0.0800,36600651.9800,12138023.8520,0.2320
5,Random Forest XLog,29336375.2420,6246264.0060,0.0820,37443336.8180,12519869.8480,0.2340
4,Random Forest Disaggregates Only,28743123.8560,6035507.3100,0.0840,37138590.8720,12468855.2040,0.2840


# Decision Tree

In [17]:
dt_results = pd.DataFrame(columns=columns)
dt_models = []
dt_vips = []

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(df_train_list)):
        dt_result, dt_model, dt_vip = decision_tree_regression(df_train_list[i], df_test_list[i], outcome_variables[i], f"Decision Tree {names[i]}", outcome_transformation = outcome_transformation[i], random_state = random_state)
        dt_results = pd.concat([dt_results, dt_result], axis=0, ignore_index=True)
        dt_models.append(dt_model)
        dt_vips.append(dt_vip)


Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                    Model Training RMSE Training MAE Training MAPE  \
0  Decision Tree Baseline   24277022.35   5034147.92          0.05   

  Testing RMSE Testing MAE Testing MAPE  
0  11146704.70  5017293.90         0.17  
Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                Model Training RMSE Training MAE Training MAPE Testing RMSE  \
0  Decision Tree XLog   24277022.35   5034147.92          0.05  11147848.30   

  Testing MAE Testing MAPE  
0  5037512.42         0.17  
Performing Decision Tree...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluating the model...


                     Model Training RMSE Training MAE Training MAPE  \
0  Decision Tree XLog YLog   24308431.12   4798586.81          0.04   

  Testing RMSE Testing MAE Testing MAPE  
0  12727551.27  6654842.7

In [18]:
dt_results.loc[:, dt_results.columns != 'Model'] = dt_results.loc[:, dt_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
dt_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
3,Decision Tree Baseline,14732790.0080,3109712.3300,0.0360,30017159.0520,11046364.9360,0.2160
5,Decision Tree XLog,14653850.4780,3098652.2000,0.0340,32852911.8800,11860650.1600,0.2180
4,Decision Tree Disaggregates Only,5082944.6140,1144066.4880,0.0240,30410442.1560,10751974.8340,0.2220
7,Decision Tree YLog,28298985.5660,6019625.5460,0.0560,53822603.4780,17095497.8600,0.2440
2,Decision Tree Aggregates Only YLog,29351417.3700,6708681.1120,0.0640,66000628.9300,21318852.9340,0.2460
1,Decision Tree Aggregates Only,16350463.3160,4012843.7580,0.0600,34796633.5400,12521636.3640,0.2580
0,Decision Tree Aggregates N1-4 N5-7,4968053.4160,1113052.9740,0.0180,35691519.4860,13040263.2280,0.2640
6,Decision Tree XLog YLog,33598548.2400,7324825.7300,0.0620,87157855.3420,28943622.3480,0.2860


# Cluster-Based Modeling

In [16]:
cluster_results = pd.DataFrame(columns=columns)
cluster_models_c0 = []
cluster_models_c1 = []

outcome_variables_cbm = ["cTOTEXn", "cTOTEXn_log"]
names_cbm = ["Baseline", "YLog"]
outcome_transformation_cbm = ["None", "log"]
indices = [0, 3]

for random_state in random_states:
    df_train, df_test = prepare_base_data(df, random_state = random_state)
    df_train_list, df_test_list = create_variations(df_train, df_test)
    for i in range(len(outcome_transformation_cbm)):
        cluster_result, model_c0, model_c1 = cluster_based_modeling(df_train_list[indices[i]], df_test_list[indices[i]], outcome_variables_cbm[i], f"Cluster-Based Modeling {names_cbm[i]}", outcome_transformation = outcome_transformation_cbm[i], random_state = random_state)
        cluster_results = pd.concat([cluster_results, cluster_result], axis=0, ignore_index=True)
        cluster_models_c0.append(model_c0)
        cluster_models_c1.append(model_c1)


Cluster
 0    104
-1     70
Name: count, dtype: int64
Performing Cluster-Based Modeling...
Performing Lasso regression...
Evaluating the model...


                             Model Training RMSE Training MAE Training MAPE  \
0  Cluster-Based Modeling Baseline     882075.12    701492.50          0.08   

  Testing RMSE Testing MAE Testing MAPE  
0   2365944.25  1909514.11         0.14  
Evaluating the model...


   Model Training RMSE Training MAE Training MAPE Testing RMSE Testing MAE  \
0  Lasso     882075.12    701492.50          0.08   2365944.25  1909514.11   

  Testing MAPE  
0         0.14  
Performing Lasso regression...
Evaluating the model...


                             Model Training RMSE Training MAE Training MAPE  \
0  Cluster-Based Modeling Baseline    2655365.01   1972121.61          0.05   

  Testing RMSE Testing MAE Testing MAPE  
0   5479753.39  4568669.98         0.21  
Evaluating the model...


   Model Training RMSE Training MAE Training MAPE Testing RMSE Tes

In [17]:
cluster_results.loc[:, cluster_results.columns != 'Model'] = cluster_results.loc[:, cluster_results.columns != 'Model'].apply(pd.to_numeric, errors='coerce')
cluster_results.groupby(by = "Model").mean().reset_index().sort_values(by = "Testing MAPE")

,Model,Training RMSE,Training MAE,Training MAPE,Testing RMSE,Testing MAE,Testing MAPE
1,Cluster-Based Modeling Baseline_Random Forest_...,3937721.2600,2388153.2100,0.0900,11803949.3000,5317071.3400,0.1300
2,Cluster-Based Modeling YLog_Lasso_Random Forest,36981423.7100,6955069.8300,0.1100,17875542.3950,8817983.6750,0.1450
0,Cluster-Based Modeling Baseline_Lasso_Lasso,2032600.5575,1410583.9525,0.0775,10604719.5700,5455368.2300,0.2400
3,Cluster-Based Modeling YLog_Random Forest_Rand...,34018988.1200,6370965.6067,0.0767,57132434.3733,16252076.9867,0.2833


In [ ]:
# chosen frequency: 1, 2, 4, 3

# Final Results

In [ ]:
# merge them all